## Генерация цифровой подписи RSA

In [43]:
alhash = [chr(i) for i in range(ord('А'), ord('Я') + 1)] 
alhash.append('Й')

In [40]:
def hash_square(i, msg, al=alhash, p=11): # i == len(msg)
    if i == 0:
        return 0
    symbol = alhash.index(msg[i - 1]) + 1
    #print(*[symbol,msg[i - 1]])
    return (( hash_square(i - 1, msg, al, p) + symbol ) ** 2 ) % p

In [41]:
msg = 'МАША'
hash_square(len(msg), msg)

5

In [6]:
from random import randint

def prime_gen():
    primes = [ 2, 3, 5, 7, 11, 13, 
        17, 19, 23, 29, 31, 37, 
        41, 43, 47, 53, 59, 61, 
        67, 71, 73, 79, 83, 89, 
        97, 101]
    a = randint(1, len(primes)-1)
    return primes[a]

def linear_congruence(a, b, m):
    # aX ≡ b (mod m)
    #assert gcd(a,m) == gcd(b,m)
    if b == 0:
        return 0
    if a < 0:
        a = -a
        b = -b
    b %= m
    while a > m:
        a -= m

    return (m * linear_congruence(m, -b, a) + b) // a

In [18]:
from math import gcd

def sign_RSA(doc, hash_func=hash_square):
    p = prime_gen()
    q = prime_gen()
    while p == q:
        q = prime_gen()
    n = p * q 
    phi = (p - 1) * (q - 1)
    
    e = randint(1, p)
    while gcd(phi, e) != 1:
        e = randint(1, p)
    d = linear_congruence(e, 1, phi)
    print(f'p:{p} q:{q} n:{n} phi:{phi} e:{e} d:{d}')
    print(hash_square(len(doc), doc))
    return ((hash_square(len(doc), doc)) ** d) % n, (e, n)

def sign_RSA_check(hashmsg, doc, hash_func=hash_square):
    hash_n, open_key = hashmsg
    dec = (hash_n ** open_key[0]) % open_key[1]
    return bool(dec == hash_square(len(doc), doc))

In [46]:
text = 'НЕВСЯКИЙЗПТКТОЧИТАЕТЗПТВЧТЕНИИСИЛУЗНАЕТТЧК'

test = sign_RSA(text)
test, sign_RSA_check(test, text)

p:29 q:71 n:2059 phi:1960 e:3 d:1307
9


((817, (3, 2059)), True)

In [48]:
import requests
from bs4 import BeautifulSoup as bs

def clear_paragraphs(ps:list, start=14, stop=16):
    return ''.join([i.text for i in ps[start:stop]]).replace('\xad', '').replace('\xa0', '')

def test_text(url='https://spacemorgue.com/matter-matters/'):
    '''
    Возвращает тестовый текст длиной 2408 символов
    '''
    soup = bs(requests.get(url).text, 'html.parser')
    return clear_paragraphs(soup.select_one('article').find_all('p'))

def prepare(text, specials={',':'ЗПТ','.':'ТЧК', ' ':'', '—':'ТИРЕ', '«':'КВЧ', '»':'КВЧ'}):
    '''
    Возвращает текст, удаляя пробельные символы в тексте и заменяя пунктуацию телеграфными обозначениями
    '''
    for i in specials.keys():
        text = text.replace(i, specials[i])
    return text

In [49]:
text = prepare(test_text().upper().replace('-', '—').replace('X', '').replace('I', ''))

In [50]:
test = sign_RSA(text)
test, sign_RSA_check(test, text)

p:7 q:79 n:553 phi:468 e:5 d:281
9


((95, (5, 553)), True)

## Генерация цифровой подписи Эль-Гамаля (ЕGSА)

In [51]:
def gen_k(p):
    k = 2
    while gcd(k, p - 2) != 1:
        k = randint(1, p-2)
    return k

def sign_EGSA(doc, hash_func=hash_square):
    p = prime_gen()
    g = randint(1, p-1)
    x = randint(1, p-1)
    y = (g ** x) % p
    k = gen_k(p)
    a = (g ** k) % p 
    b = linear_congruence(k,(hash_square(len(doc), doc) - x * a),p-1)
    return (a, b), (p, g, y)

def sign_EGSA_check(hashmsg, doc, hash_func=hash_square):
    open_key, hash_n = hashmsg
    a, b = open_key
    p, g, y = hash_n
    a_1 = ((y ** a) * (a ** b)) % p
    a_2 = (g ** hash_square(len(doc), doc)) % p
    return bool(a_1 == a_2)

In [62]:
text = 'НЕВСЯКИЙЗПТКТОЧИТАЕТЗПТВЧТЕНИИСИЛУЗНАЕТТЧК'

test = sign_EGSA(text)
test, sign_EGSA_check(test, text)

(((3, 2), (23, 16, 8)), True)

In [56]:
text = 'ТЕСТ'

test = sign_EGSA('ТЕСТ')
test, sign_EGSA_check(test, text)

(((55, 19), (89, 12, 12)), True)

In [63]:
text = prepare(test_text().upper().replace('-', '—').replace('X', '').replace('I', ''))
test, sign_EGSA_check(test, text)

(((3, 2), (23, 16, 8)), True)